In [ ]:
# !pip3 install pandas matplotlib

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

Time-decay модель.
Веса для модели расчитываются по формуле:

вес сессии = $2^{- d / s}$

где d — дней до покупки
s — количество сессий у пользователя

Почему такая формула? Давайте посмотрим на график функции

y = $2^{- x / s}$

Мы хотим увидеть, как меняется вес сессии в зависимости от 2 факторов:
- времени до сессии с покупкой
- количества сессий у пользователя

График выглядит так:

In [ ]:
x = list(range(1, 31, 3))

plt.figure(figsize=(10, 6))

for s in range(6, 16, 3):
    w = [2**(-d / s) for d in x]
    y = [weight / sum(w) for weight in w]  # нормируем, чтобы сумма весов была равна 1
    plt.plot(x, y, '-o', label=f'{s} сессий')

plt.legend(loc='best')
plt.xlabel('Количество дней до момента покупки')
plt.ylabel('Вес сессии')
plt.grid()
plt.show()

По графику можно сделать следующие выводы:
- график — убывающая функция: чем больше дней до покупки (горизонтальная ось), тем меньше вес у соответствующей сессии
- если пользователь приходил нечасто, то есть у него мало сессий, то веса очень сильно убывают с течением времени
- если пользователь часто к нам приходил, то веса убывают относительно медленно

Именно для того, чтобы функция убывала с ростом количества дней до покупки, перед количеством дней в формуле стоит знак «минус».
А для того, чтобы регулировать скорость убывания функции, мы используем количество сессий в качестве делителя.

Важный вывод — если пользователь приходит к нам редко, то для нас очень ценны сессии непосредственно перед покупкой. И наоборот, давние сессии для редкого гостя имеют мало ценности для бизнеса.

In [ ]:
# если код выполняется в colab-ноутбуке, то замените первую строчку в следующей ячейке на эту:
# session_data = pd.read_csv('https://drive.google.com/file/d/1y1qm5nZFb89Muroe3nhcgboSTSX3ZJLi/view?usp=sharing')

In [ ]:
session_data = pd.read_csv('data.csv', delimiter=';')

# изменение форматов данных для удобства
session_data['date'] = [datetime.strptime(x, '%Y-%m-%d') for x in session_data['date']]
session_data[['cost', 'value']] = session_data[['cost', 'value']].astype(float)

# группировка и суммирования, чтобы объединить покупки с одинакового канала в один день
session_data = session_data.groupby(['userId', 'date', 'trafficSource'])['cost', 'value'].sum().reset_index()
session_data = session_data.sort_values(by=['userId', 'date'])
session_data.head()

In [ ]:
# таблица только с датами покупок, нужна для time-decay модели

purchases_only = session_data[session_data['value'] > 0][['userId', 'date']]
purchases_only = purchases_only.groupby('userId')['date'].max().reset_index()
purchases_only.columns = ['userId', 'purchaseDate']

In [ ]:
# расчет вспомогательных колонок

session_data['totalSessions'] = session_data.groupby('userId')['date'].transform(lambda x: x.count())
session_data['totalValue'] = session_data.groupby('userId')['value'].transform(lambda x: x.sum())
session_data['sessionNumber'] = session_data.groupby('userId').cumcount() + 1

session_data = session_data.merge(purchases_only, on='userId', how='left')

In [ ]:
# расчет весов time-decay модели

session_data['daysToPurchase'] = [(x - y).days if x else 0
                                  for x, y in zip(session_data['purchaseDate'], session_data['date'])]

session_data['timeDecayWeight'] = [2**(-x / y)
                                   for x, y in zip(session_data['daysToPurchase'], session_data['totalSessions'])]

session_data['timeDecayWeight'] = session_data['timeDecayWeight'] / session_data.groupby('userId')['timeDecayWeight'].transform(lambda x: x.sum())


In [ ]:
# расчет выручки на основе различных моделей атрибуции

session_data['lastTouchValue'] = session_data['value']
session_data['firstTouchValue'] = [x if y == 1 else 0
                                   for x, y in zip(session_data['totalValue'], session_data['sessionNumber'])]
session_data['linearValue'] = session_data['totalValue'] / session_data['totalSessions']
session_data['timeDecayValue'] = session_data['totalValue'] * session_data['timeDecayWeight']


In [ ]:
# проверка корректности расчетов

print(session_data[['lastTouchValue', 'firstTouchValue', 'linearValue', 'timeDecayValue']].sum())

In [ ]:
# финальная таблица

totals = session_data.groupby('trafficSource')[['cost', 'lastTouchValue', 'firstTouchValue', 'linearValue',
                                                'timeDecayValue']].sum()

totals['lastTouchROI'] = 100*(round(totals['lastTouchValue'] / totals['cost'], 4))
totals['firstTouchROI'] = 100*(round(totals['firstTouchValue'] / totals['cost'], 4))
totals['linearROI'] = 100*(round(totals['linearValue'] / totals['cost'], 4))
totals['timeDecayROI'] = 100*(round(totals['timeDecayValue'] / totals['cost'], 4))

totals[['cost', 'lastTouchROI', 'firstTouchROI', 'linearROI', 'timeDecayROI']]